### GENERATE_DH

In [16]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives import serialization

class DH_pair:
    def __init__(self, dh_priv, dh_pub) -> None:
        self.dh_priv = dh_priv
        self.dh_pub = dh_pub

def GENERATE_DH():
    # Generate a new DH pair key.
    dh_priv = X25519PrivateKey.generate()
    dh_pub = dh_priv.public_key()
    
    return DH_pair(dh_priv, dh_pub)

dh_pair = GENERATE_DH()
dh_pair2 = GENERATE_DH()

print("Private key:", dh_pair.dh_priv.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption()))
print("Length Private key:", len(dh_pair.dh_priv.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption())))
print()
print("Public key:", dh_pair.dh_pub.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw))
print("Length Public key:", len(dh_pair.dh_pub.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)))


Private key: b'X\x19\x81\xcb\xcf\xca\x97\t\x00[A<\xe5b\xab[\\z\x19P\x07|\x1d\xc6\xa9\xca\xe6Y\xd1\xba\xaf{'
Length Private key: 32

Public key: b'\x93\xea\xf5\x1c\xdf1.\x99.\xf8?\xc1\xcc\xe5(K\xef\xc2\xc2\x08m\xc70P\x07\xd8\xec-\x99X\xa6-'
Length Public key: 32


### DH

In [17]:
def DH(dh_pair: DH_pair, dh_pub):
    # Perform DH calculation between private key and public key
    return dh_pair.dh_priv.exchange(dh_pub)

# Calcular la clave compartida usando la función DH()
dh_out = DH(dh_pair=dh_pair, dh_pub=dh_pair2.dh_pub)
dh_out2 = DH(dh_pair=dh_pair2, dh_pub=dh_pair.dh_pub)
print("Shared secret 1:", dh_out)
print("Shared secret 2:", dh_out2)

Shared secret 1: b'\x1f\xd5$kMQ\xb1\x99\xb1\xe4Y\x01\x86\x1c\x04\x90U\xf3\\\xd9t\x82"J\xa6\xdf\x92w\xb8\x06\xf4\x01'
Shared secret 2: b'\x1f\xd5$kMQ\xb1\x99\xb1\xe4Y\x01\x86\x1c\x04\x90U\xf3\\\xd9t\x82"J\xa6\xdf\x92w\xb8\x06\xf4\x01'


### KDF_RK

In [18]:
import secrets

rk = secrets.token_bytes(32)
print("Random",len(rk),"bytes:", rk)

Random 32 bytes: b'B\xcf\x19g\x07\xe3\x17\xdbb\x1c\x93f\xebV1vL\xaf\x95\x1c\x16\x05k\x166\x82YX\x13\xe0\xad\x00'


In [19]:
from hkdf import hkdf_expand, hkdf_extract
from binascii import unhexlify
import hashlib

def KDF_RK(rk: bytes, dh_out: bytes) -> [[bytes]*32, [bytes]*32]:
    prk = hkdf_extract(salt=rk, input_key_material=dh_out, hash=hashlib.sha512)
    key = hkdf_expand(pseudo_random_key=prk, info=b"Info KDF_RK", length=64, hash=hashlib.sha512)

    return key[:32],key[32:] # Root Key, Chain Key

rk, ck = KDF_RK(rk=rk, dh_out=dh_out)
print("Root Key:", rk)
print("Length Root Key:", len(rk), "\n")
print("Chain Key:", ck)
print("Length Chain Key:", len(ck))

Root Key: b'H\xbb\x16\x1f\xd2\x1f\xaa\xe2~\x1d[\xa8\x8a\xbaZ\xc4\xf5C\xba[V\xb6\xe2\xe8HG\x1f\x14\xb9\xe0\xea\x16'
Length Root Key: 32 

Chain Key: b"'\xfa@\x15\xcck+\xa0\x94\xe6M\x14\xeaPh|`\xa1\xb49\x9cW\xa3\xf6,\x88\x97w\xb0bJ\x84"
Length Chain Key: 32


### KDF_CK

In [20]:
import hmac

def KDF_CK(ck: bytes) -> [[bytes]*32, [bytes]*32]:
    new_ck = hmac.new(key=ck, msg=b'Chain Key', digestmod=hashlib.sha512)
    mk = hmac.new(key=ck, msg=b'Message Key', digestmod=hashlib.sha512)

    return new_ck.digest()[:32], mk.digest()[:32] # Chain Key, Message Key


ck, mk = KDF_CK(ck=ck)

print("Chain Key:",ck)
print("Length Chain Key:",len(ck))
print()
print("Message Key:",mk)
print("Length Message Key:",len(mk))


Chain Key: b'\xd0\xd7+i\xf2o\xbc\xfc\xa5\xebT\x1d<q7r\xe4\x15\x1b+\xc0W\xc3\x94\xbc\xfa\xec\x85\x06CR\xfe'
Length Chain Key: 32

Message Key: b'\xbd\xbf(F\xa2%.\xf8*\xe9\xaci\xdc\xa7\x94\xb7~(\x96\x86SR\xac\xc3\xbfk\xf6Ox\x04\x1d\xa9'
Length Message Key: 32


### ENCRYPT

In [21]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding  # - Import to use the padding.
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

def ENCRYPT(mk: bytes, plaintext: bytes, assotiate_data: bytes):
    # Key Derivation Function
    output_length = 80
    prk = hkdf_extract(salt=bytes(output_length), input_key_material=mk, hash=hashlib.sha512)
    exp = hkdf_expand(pseudo_random_key=prk, info=b"Info ENCRYPT/DECRYPT", length=output_length, hash=hashlib.sha512)

    sk = exp[:32] # Encryption Key
    ak = exp[32:64] # Authentication Key
    iv = exp[64:] # Initialization vector

    # Construct an AES-128-GCM Cipher object with the generated encryption key and IV.
    encryptor = Cipher(
            algorithms.AES(key=sk),
            modes.GCM(initialization_vector=iv),
            backend=default_backend()
        ).encryptor()

    padder = padding.PKCS7(128).padder()
    padder_data = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padder_data) + encryptor.finalize()
    tag = encryptor.tag # Tag is always 16B long.

    # HMAC for authentication of the associate it data.
    ad = hmac.new(key=ak, msg=assotiate_data, digestmod=hashlib.sha512).digest() # 512b = 64B

    return tag + ad + ciphertext

enc_msg = ENCRYPT(mk=mk,plaintext=b"hola",assotiate_data=b"hola")
print(enc_msg)
print(len(enc_msg))

b"9\xe3\x9b\xbb\x99\x92h\x01\xfb\xd9\xbe\xb1\x95\x03\xf3z\xb0yRO\x9d\xd6\xa1\xef\xce\xc2\x87\x13A'\xd1\xc6T\x94\t\xcd\x8a\xe7S\xa0\x860q\xc1\xf6b\x05\x9fk\xe1m\xc4`S\xc4\x05sx\xb2\xbf%vD\xd6\x0eS\xc1\xac\x02\xbcwC\xa3\xc3\xe5\x10\xe2\x87\xa3\x96(\xf8\x16\xfe\xf3\xc7\xa44L\n$\x06ow\xbaO"
96


### HEADER

In [22]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PublicKey

class HEADER:
    def __init__(self, dh_pair=None, pn: int = 0, n: int = 0) -> None:
        if dh_pair is not None:
            self.dh = dh_pair.dh_pub
            self.pn = pn
            self.n = n

    def to_bytes(self):
        dh_pub_b = self.dh.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw) # 32 B
        pn_b = self.pn.to_bytes(1, byteorder='big') # 1 B
        n_b = self.n.to_bytes(1, byteorder='big') # 1 B

        return dh_pub_b + pn_b + n_b 
    
    def to_object(self, header_bytes: [bytes]*34):
        self.dh = X25519PublicKey.from_public_bytes(data=header_bytes[:32])
        self.pn = int.from_bytes(header_bytes[32:33], byteorder='big')
        self.n = int.from_bytes(header_bytes[33:34], byteorder='big')

header = HEADER(dh_pair=dh_pair, pn=4, n=3)
print(header.to_bytes())
print(header)

b'\x93\xea\xf5\x1c\xdf1.\x99.\xf8?\xc1\xcc\xe5(K\xef\xc2\xc2\x08m\xc70P\x07\xd8\xec-\x99X\xa6-\x04\x03'


In [23]:


#dh_pub = X25519PublicKey.from_public_bytes(data=header[:32])

### CONCAT

In [24]:
def CONCAT(ad: bytes, header: HEADER):
    return ad + header.to_bytes()

ad = b"Hola"
ad = hashlib.sha256(ad).digest()
print(len(ad))
concat = CONCAT(ad=ad, header=header)
print(concat)
h2 = HEADER()
print(type(h2))
h2.to_object(header_bytes=concat[32:])
h2.n

32
b'\xe63\xf4\xfcy\xba\xde\xa1\xdc]\xb9p\xcf9|\x82H\xba\xc4|\xc3\xac\xf9\x91[\xa6\x0b]v\xb0\xe8\x8f\x93\xea\xf5\x1c\xdf1.\x99.\xf8?\xc1\xcc\xe5(K\xef\xc2\xc2\x08m\xc70P\x07\xd8\xec-\x99X\xa6-\x04\x03'
<class '__main__.HEADER'>


3

### MESSAGE TO SEND

In [25]:
header = HEADER(dh_pair=dh_pair,pn=0,n=0)#34B
ad = b"Associate data"
ad = hashlib.sha256(ad).digest()#32B
enc_msg = ENCRYPT(mk=mk, plaintext=b"Hola mundo!",assotiate_data=CONCAT(ad, header=header))
header, enc_msg

(<__main__.HEADER at 0x7f6d8df828f0>,
 b'\xfeCq\xbc\x06\xc8\x05\x94\xb2\xc9\xd9\x1c\xe9\xa9%H\xefh"\x9c1\xd0\x93\xe6\xcfR\xe5+!\xe1\xe0\xbc7\xebZ\xadNl\x85\xc0{s\x83\xf4\xf40F,J\xefI\x19?`{\xd0\xc9Jth4\n\xdeb\t\xc9\x83x\xdd\xa8\xa7\xecD\xba\x85\x18&6\xa3\xd1\x08\xf8\x16\xfe\xdf\xa6\xddV$i\t\x0ff~\xb3F')

### DECRYPT

In [26]:
def DECRYPT(mk, ciphertext, assotiate_data):
    tag = ciphertext[:16]
    ad_original = ciphertext[16:16+64]
    ciphertext = ciphertext[16+64:]

    # Key Derivation Function
    output_length = 80
    prk = hkdf_extract(salt=bytes(output_length), input_key_material=mk, hash=hashlib.sha512)
    exp = hkdf_expand(pseudo_random_key=prk, info=b"Info ENCRYPT/DECRYPT", length=output_length, hash=hashlib.sha512)

    sk = exp[:32] # Encryption Key
    ak = exp[32:64] # Authentication Key
    iv = exp[64:] # Initialization vector

    decryptor = Cipher(
        algorithms.AES(key=sk),
        modes.GCM(initialization_vector=iv, tag=tag),
        backend=default_backend()
    ).decryptor()

    # Decryption and unpadding.
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(plaintext) + unpadder.finalize()

    # HMAC for authentication of the associate it data.
    ad = hmac.new(key=ak, msg=assotiate_data, digestmod=hashlib.sha512).digest() # 512b = 64B

    if ad != ad_original:
        raise Exception("Error authenticating data.")
    
    return plaintext

DECRYPT(mk=mk, ciphertext=enc_msg, assotiate_data=CONCAT(ad=ad, header=header))

b'Hola mundo!'

### THREADS

In [27]:
from threading import Thread

class Hola:
    def __init__(self) -> None:
        self.hola = "hola"
h = Hola()

def say_hola(h3):
    print(h3.hola)

h2 = Hola()
h2.hola = "adios"
t_publisher = Thread(target=say_hola, args=(h2,))
t_publisher.start()

adios


In [28]:
import paho.mqtt.client as mqtt
from threading import Thread

# MQTT broker address and port
mqttBroker = '18.101.47.122'
password = 'HkxNtvLB3GC5GQRUWfsA'
mqtt_id = 'sinf'
port = 1883

def publisher():
    client = mqtt.Client()
    client.username_pw_set(username=mqtt_id, password=password)
    client.connect(mqttBroker)

    while True:
        m = input("\nMensaje: ")
        client.publish(m, "mqtt.in")
        print("\nJust published " + m + " to topic.")

def on_message(client, userdata, message):
    print(message.payload())

def subscriber():
    client = mqtt.Client()
    client.username_pw_set(username=mqtt_id, password=password)
    client.connect(mqttBroker)

    client.subscribe("mqtt.in")
    client.on_message = on_message
    client.loop_forever()

t_publisher = Thread(target=publisher)
t_subscriber = Thread(target=subscriber)
t_publisher.start()
t_subscriber.start()

In [29]:
a = b"hola"
print(a.decode())
a = "hola"
a.encode()

hola


b'hola'

In [30]:
class hola:
    def __init__(self) -> None:
        return
    
    def a(self, a):
        print(a)
        self.b(self,"adios")

    def b(self, b):
        print(b)

h = hola()
h.a("hi")

hi


TypeError: hola.b() takes 2 positional arguments but 3 were given

Exception in thread Thread-5 (publisher):
Traceback (most recent call last):
  File "/home/pedro/anaconda3/envs/munics/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/pedro/anaconda3/envs/munics/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_6445/3826392129.py", line 17, in publisher
  File "/home/pedro/anaconda3/envs/munics/lib/python3.10/site-packages/paho/mqtt/client.py", line 1228, in publish
    raise ValueError('Invalid topic.')
ValueError: Invalid topic.


In [ ]:
sk: b'\x16\xd7\xc5\x0c\xb2\xe8\xc4\x02\xae\xb7{}fE\x98\x06\n\x88\xf9XU\xed.@\xb6RV\x13F\xc0\xf0\xac'
ak: b'1\xde7\xa8Hy" \xc00\xf7\xaen\x8d\xfeG\xadE\xa2\xcc\xecoG\x8e\x7f\x98\xf7c\x0f\xd6e\xab'
iv: b'\x95\xf3\xfa\x9b\xa2\xac#kTy\\\x98\x8f6\xbc\xf5'

sk: b'.[\x16E]w=\xd0\xe4Tyi$>v\x9d\x1e\xc9\x19\xd3mj\xee1\x9bbF$\x9d\xb5"G'
ak: b'h\x90\xec\xa1"\x1d\x86\t\x0e9\x13\xfb\x07\xd8q\x02e\xf7l\xb5\x07mC?s+8?\xf8\xffI\xf1'
iv: b"~\xcbR)\x86\x92\n\x99\n#'\xab\x85b\xf4\x8e"